<a href="https://colab.research.google.com/github/kiril-buga/Neural-Network-Training-Project/blob/main/ECG_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# ===== Install required libraries (run once per session) =====
!pip install wfdb
!pip install ecg-qc

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import wfdb
import matplotlib.pyplot as plt # for plotting

In [3]:
# ===== Detect if running in Google Colab and mount Drive =====
IN_COLAB = False
try:
    from google.colab import drive  # type: ignore
    IN_COLAB = True
except Exception:
    drive = None
    IN_COLAB = False

if IN_COLAB:
    drive.mount('/content/drive/')

# ===== Define paths =====
if IN_COLAB:
    # Case 1: You manually placed the dataset in MyDrive
    DATA_PATH = "/content/drive/MyDrive/DeepLearningECG/data/"
    ARTIFACT_DIR = "/content/drive/MyDrive/DeepLearningECG/artifacts/"

else:
    # Case 3: Local fallback (if running outside Colab)
    DATA_PATH = "../data/"
    ARTIFACT_DIR = "../artifacts/"

print("DATA_PATH:", DATA_PATH)
print("ARTIFACT_DIR:", ARTIFACT_DIR)
print("Files in DATA_PATH:", os.listdir(DATA_PATH))

Mounted at /content/drive/
DATA_PATH: /content/drive/MyDrive/DeepLearningECG/data/
ARTIFACT_DIR: /content/drive/MyDrive/DeepLearningECG/artifacts/
Files in DATA_PATH: ['ECGCode.csv', 'DiseaseCode.csv', 'ExampleReadingCode.ipynb', 'AttributesDictionary.csv', 'Child_ecg.zip', 'Child_ecg.z01', 'Child_ecg']


In [6]:
# load CSV
df_attr = pd.read_csv(DATA_PATH + 'AttributesDictionary.csv')
df_attr

,Filename,ECG_ID,Patient_ID,Age,Gender,Acquisition_date,Sampling_point,Lead,AHA_code,CHN_code,ICD-10 code,pSQI,basSQI,bSQI
0,P00/P00001/P00001_E01,P00001_E01,P00001,572d,'Female',2017-11-22 10:46:08,9000,9,'Left ventricular high voltage';'L147','J106';'L123','I34.0';'Q21.0';'Q24.9','I':0.288;'II':0.323;'III':0.346;'aVR':0.312;'...,'I':0.994;'II':0.996;'III':0.991;'aVR':0.997;'...,'I':1.000;'II':1.000;'III':1.000;'aVR':1.000;'...
1,P00/P00002/P00002_E01,P00002_E01,P00002,4327d,'Male',2017-11-28 21:59:47,15000,12,'C21','C13','I51.4';'J18.9','I':0.472;'II':0.446;'III':0.449;'aVR':0.484;'...,'I':0.995;'II':0.980;'III':0.992;'aVR':0.992;'...,'I':1.000;'II':1.000;'III':1.000;'aVR':1.000;'...
2,P00/P00003/P00003_E01,P00003_E01,P00003,1087d,'Female',2017-11-29 16:04:57,10000,12,'C21','C13','Q21.0';'Q24.9','I':0.495;'II':0.347;'III':0.340;'aVR':0.382;'...,'I':0.915;'II':0.895;'III':0.882;'aVR':0.908;'...,'I':1.000;'II':1.000;'III':1.000;'aVR':1.000;'...
3,P00/P00004/P00004_E01,P00004_E01,P00004,2465d,'Male',2017-11-30 15:21:27,13000,9,'C21','C13','Q21.1';'Q24.9','I':0.340;'II':0.405;'III':0.409;'aVR':0.350;'...,'I':0.981;'II':0.988;'III':0.974;'aVR':0.986;'...,'I':1.000;'II':1.000;'III':1.000;'aVR':1.000;'...
4,P00/P00004/P00004_E02,P00004_E02,P00004,2461d,'Male',2017-11-26 19:19:48,15000,9,'A1','A1','Q21.1';'Q24.9','I':0.501;'II':0.494;'III':0.389;'aVR':0.525;'...,'I':0.993;'II':0.993;'III':0.989;'aVR':0.995;'...,'I':1.000;'II':1.000;'III':1.000;'aVR':1.000;'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14185,P11/P11639/P11639_E01,P11639_E01,P11639,2646d,'Male',2021-06-24 18:22:31,10000,12,'A1','A1','J35.3','I':0.330;'II':0.422;'III':0.387;'aVR':0.377;'...,'I':0.991;'II':0.991;'III':0.981;'aVR':0.992;'...,'I':1.000;'II':1.000;'III':0.990;'aVR':1.000;'...
14186,P11/P11640/P11640_E01,P11640_E01,P11640,657d,'Male',2021-07-01 09:47:16,10500,12,'C21';'L147','C13';'L123','S02.0';'S06.5';'S06.6';'S06.7';'T14.0','I':0.284;'II':0.362;'III':0.378;'aVR':0.332;'...,'I':0.919;'II':0.934;'III':0.939;'aVR':0.929;'...,'I':0.976;'II':0.993;'III':1.000;'aVR':0.993;'...
14187,P11/P11641/P11641_E01,P11641_E01,P11641,1484d,'Female',2021-07-04 21:58:36,15000,12,'D30+Modifier310','D21+Frequent','I49.1';'R53','I':0.387;'II':0.387;'III':0.411;'aVR':0.384;'...,'I':0.985;'II':0.975;'III':0.960;'aVR':0.980;'...,'I':0.994;'II':0.987;'III':0.982;'aVR':0.994;'...
14188,P11/P11642/P11642_E01,P11642_E01,P11642,5178d,'Male',2021-06-27 20:22:00,15000,12,'C23';'L150','C15';'L128','J31.0';'J34.2';'S02.2','I':0.401;'II':0.409;'III':0.409;'aVR':0.407;'...,'I':0.975;'II':0.973;'III':0.974;'aVR':0.973;'...,'I':0.974;'II':1.000;'III':1.000;'aVR':1.000;'...


In [7]:
# ===== Load ECG data =====
# Path where the WFDB ECG files (.hea/.dat) live.
ECG_PATH = os.path.join(DATA_PATH, "Child_ecg")

# Load the raw ECG signal data
def load_raw_data(df, path, n_samples=None):
    """Return list of ECG arrays and list of metadata dicts."""
    filenames = df["Filename"].tolist()
    if n_samples is not None:
        filenames = filenames[:n_samples]

    signals = []
    metas = []
    for fname in filenames:
        sig, meta = wfdb.rdsamp(os.path.join(path, fname))
        signals.append(sig)
        metas.append(meta)
    return signals, metas

# Load diagnostic comments from WFDB metadata
def load_Diag(df, path, n_samples=None):
    """Return disease and ECG diagnostic comments from WFDB records."""
    filenames = df["Filename"].tolist()
    if n_samples is not None:
        filenames = filenames[:n_samples]

    disease_diag = []
    ecg_diag = []
    for fname in filenames:
        record = wfdb.rdrecord(os.path.join(path, fname))
        comments = record.comments
        disease_diag.append(comments[1] if len(comments) > 1 else None)
        ecg_diag.append(comments[2] if len(comments) > 2 else None)
    return disease_diag, ecg_diag

# ===== Load a few ECG records and check shapes =====

# Load only a few records so Colab RAM is safe
N_SAMPLES = 5
signals, metas = load_raw_data(df_attr, ECG_PATH, n_samples=N_SAMPLES)

print(f"Loaded {len(signals)} ECG signals.")
print("Shape of first signal (time, leads):", signals[0].shape)
print("Meta of first signal:")
print(metas[0])

# Attach comments for the same subset
disease_diag, ecg_diag = load_Diag(df_attr, ECG_PATH, n_samples=N_SAMPLES)

df_subset = df_attr.iloc[:N_SAMPLES].copy()
df_subset["Disease_diag_comment"] = disease_diag
df_subset["ECG_diag_comment"] = ecg_diag
display(df_subset)


Loaded 5 ECG signals.
Shape of first signal (time, leads): (9000, 9)
Meta of first signal:
{'fs': 500, 'sig_len': 9000, 'n_sig': 9, 'base_date': None, 'base_time': None, 'units': ['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV'], 'sig_name': ['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V3', 'V5'], 'comments': ['Female 572', 'Ventricular septal defect', 'Left ventricular high voltage;T-wave abnormality']}


,Filename,ECG_ID,Patient_ID,Age,Gender,Acquisition_date,Sampling_point,Lead,AHA_code,CHN_code,ICD-10 code,pSQI,basSQI,bSQI,Disease_diag_comment,ECG_diag_comment
0,P00/P00001/P00001_E01,P00001_E01,P00001,572d,'Female',2017-11-22 10:46:08,9000,9,'Left ventricular high voltage';'L147','J106';'L123','I34.0';'Q21.0';'Q24.9','I':0.288;'II':0.323;'III':0.346;'aVR':0.312;'...,'I':0.994;'II':0.996;'III':0.991;'aVR':0.997;'...,'I':1.000;'II':1.000;'III':1.000;'aVR':1.000;'...,Ventricular septal defect,Left ventricular high voltage;T-wave abnormality
1,P00/P00002/P00002_E01,P00002_E01,P00002,4327d,'Male',2017-11-28 21:59:47,15000,12,'C21','C13','I51.4';'J18.9','I':0.472;'II':0.446;'III':0.449;'aVR':0.484;'...,'I':0.995;'II':0.980;'III':0.992;'aVR':0.992;'...,'I':1.000;'II':1.000;'III':1.000;'aVR':1.000;'...,Myocarditis,Sinus tachycardia
2,P00/P00003/P00003_E01,P00003_E01,P00003,1087d,'Female',2017-11-29 16:04:57,10000,12,'C21','C13','Q21.0';'Q24.9','I':0.495;'II':0.347;'III':0.340;'aVR':0.382;'...,'I':0.915;'II':0.895;'III':0.882;'aVR':0.908;'...,'I':1.000;'II':1.000;'III':1.000;'aVR':1.000;'...,Ventricular septal defect,Sinus tachycardia
3,P00/P00004/P00004_E01,P00004_E01,P00004,2465d,'Male',2017-11-30 15:21:27,13000,9,'C21','C13','Q21.1';'Q24.9','I':0.340;'II':0.405;'III':0.409;'aVR':0.350;'...,'I':0.981;'II':0.988;'III':0.974;'aVR':0.986;'...,'I':1.000;'II':1.000;'III':1.000;'aVR':1.000;'...,Atrial septal defect,Sinus tachycardia
4,P00/P00004/P00004_E02,P00004_E02,P00004,2461d,'Male',2017-11-26 19:19:48,15000,9,'A1','A1','Q21.1';'Q24.9','I':0.501;'II':0.494;'III':0.389;'aVR':0.525;'...,'I':0.993;'II':0.993;'III':0.989;'aVR':0.995;'...,'I':1.000;'II':1.000;'III':1.000;'aVR':1.000;'...,Atrial septal defect,Normal ECG


In [8]:
X = load_raw_data(df_attr, ECG_PATH)
data = X[0].T
for i in range(data.shape[0]):
    plt.plot(data[i])

KeyboardInterrupt: 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sqi_score.ecg_qc.ecg_qc import EcgQc  # https://github.com/Aura-healthcare/ecg_qc
from biosppy.signals import ecg
import neurokit2 as nk
def get_rpeaks(signal, sample_rate=100, method="nabian2018"):
    _, info = nk.ecg_peaks(signal, sampling_rate=sample_rate, method=method, correct_artifacts=False)
    rpeaks = info['ECG_R_Peaks']
    return rpeaks
sqi = []
bsqi = []
for item in range(data.shape[0]):
    rpeaks1 = get_rpeaks(data[item], sample_rate=100, method='nabian2018')
    rpeaks2 = get_rpeaks(data[item], sample_rate=100, method='promac')
    bSQI = ecg.bSQI(rpeaks1,rpeaks2,fs=100)
    bsqi.append(bSQI)
    ecg_list = data[item].tolist()
    flattened_ecg = np.ravel(ecg_list)
    warnings.filterwarnings("ignore")
    ecg_qc = EcgQc('rfc_norm_2s.pkl', sampling_frequency=500, normalized=True)
    sqi_scores = np.array(ecg_qc.compute_sqi_scores(flattened_ecg))
    sqi.append(sqi_scores[0])
sqi = np.array(sqi)
bsqi = np.array(bsqi)
mean_sqi = np.mean(sqi,axis=0)
mean_bsqi = np.mean(bsqi,axis=0)
print('The quality of signal: pSQI = {:.3f}, basSQI = {:.3f}, bSQI = {:.3f}'.format(mean_sqi[4], mean_sqi[5], mean_bsqi))

In [10]:
# load Disease_Diag and ECG_Diag
Disease_Diag, ECG_Diag = load_Diag(df_attr, ECG_PATH)
Disease_Diag, ECG_Diag

KeyboardInterrupt: 

In [ ]:
# Split data into train val and test
# Get unique Patient_ID values
patient_ids = df_attr['Patient_ID'].unique()

# Split Patient_IDs into training, validation, and test sets
train_ids, test_ids = train_test_split(patient_ids, test_size=0.2, random_state=42)
train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Get indices for training, validation, and test sets based on Patient_ID
train_idx = df_attr[df_attr['Patient_ID'].isin(train_ids)].index
val_idx = df_attr[df_attr['Patient_ID'].isin(val_ids)].index
test_idx = df_attr[df_attr['Patient_ID'].isin(test_ids)].index

# Split X into training, validation, and test sets based on the indices
# Train
X_train = [X[i] for i in train_idx]
Y_train = [Disease_Diag[i] for i in train_idx]
# val
X_val = [X[i] for i in val_idx]
Y_val = [Disease_Diag[i] for i in val_idx]
# test
X_test = [X[i] for i in test_idx]
Y_test = [Disease_Diag[i] for i in test_idx]
print(f'The training set has {len(X_train)} records')
print(f'The val set has {len(X_val)} records')
print(f'The test set has {len(X_test)} records')